In [1]:
## Expected render times on kaggle w/ a P100
# Standard: 5-8 mins/second of footage
# Using a first pass: +25 mins/second of footage
# Lowering ffmpeg settings (higher crf, faster preset, etc.): Marginal improvement, -0.5 to -1 min/second of footage
# Rife scale: 0.25 -> -1min/second of footage
# Rife scale: 1 -> +3-4min/second of footage (unlikely to work on kaggle) 

"""
Adding your own videos:
1) Go to Input -> Upload. Upload your videos in a zip folder. 
2) Click Create a new dataset. Give it a name.
2) Save and it should automatically add to your datasets.
3) Set the variable input_zip to the correct path. ("/kaggle/input/name"), where name = the dataset name
"""

#Input settings
input_zip = "/kaggle/input/mydataset" #Can be folder or .zip. This is the path to the dataset.
delete_temporary_folders = False

#First pass
firstfilter = "none" #"realesgran", "codeformer", "none"; "none" is SIGNIFICANTLY FASTER, abt. 6x faster than "realesgran" + 0.5 downscale.
downscale_ratio = 0.5 #downscales after first pass by this much for real esgran, i.e. effective = 4x*downscale_ratio.
CODEFORMER_FIDELITY = 0.7

#Second Pass
filter = "realesrgan"
model = "realesrgan-plus"
scale = 4 #atm only works for 4x or 1x (skips this second pass), input should be atleasst 960px on the larger side to get complete benefit

#FFMPEG settings
codec = "libx265" #Only used preliminarily. Final output will always be in libx264
preset = "slow" 
pixfmt = "auto"
bitrate = 0 
crf = 18 
loglevel = "debug" 
TARGET_RES = 2160  #Reduce this if you see blotches. This is the resolution in pixels of the minimum side. Kaggle at most supports 4096x4096

#RIFE settings
FPS = 60
rife_scale = 0.5 #Higher -> more quality, lower -> if CUDA out of memory error for RIFE

#NTFY settings
ntfy_channel = None #"abcde" #For push notifications

print("Set")

## Debug
import time
all_folder_created = []
timestamps = []
timestamps.append(int(time.time()))
all_folder_created.append("Start")

import shutil
if delete_temporary_folders:
    def rmtree_safe(path):
        try:
            shutil.rmtree(path)
        except Exception as e:
            print("ERR: Could not delete path as: ")
            print(e)
else:
    rmtree_safe = lambda x:x

Set


# Unzip

In [2]:
import zipfile
import os
import shutil

if input_zip.endswith('.zip'):
    extract_to_path = "/kaggle/working/input_videos_raw"
    os.makedirs(extract_to_path, exist_ok=True)
    
    with zipfile.ZipFile(input_zip, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)
    
    print(f"Files extracted to {extract_to_path}")
else:
    shutil.copytree(input_zip, "/kaggle/working/input_videos_raw")

folder_path = "/kaggle/working/input_videos_raw/"
file_paths = [folder_path + f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

timestamps.append(int(time.time()))
all_folder_created.append("Finished unzip")
print("FINISHED UNZIP!!")

FINISHED UNZIP!!


# First pass

In [3]:

if firstfilter == "codeformer":
    # Installation
    !pip install ffmpeg-python
    %cd /content
    !rm -rf CodeFormer
    !git clone https://github.com/TitanStar73/CodeFormer.git
    %cd CodeFormer
    !pip install -r requirements.txt
    !python basicsr/setup.py develop
    
    !python scripts/download_pretrained_models.py facelib
    !python scripts/download_pretrained_models.py CodeFormer
    
    import os
    os.makedirs("/kaggle/working/codeformeupscaled",exist_ok=True)
    os.makedirs("/kaggle/working/tempcodeform",exist_ok=True)
    print("Starting codeformer")
    
    with open("/kaggle/working/codeformeupscaled/test.txt","w+") as f:
        f.write("")
        
    
    file_path = file_paths[0]
    input_file = file_path
    output_file = file_path.replace("/kaggle/working/input_videos_raw","/kaggle/working/codeformeupscaled")
    UPSCALE = 2
    os.system(f'python inference_codeformer.py -w {CODEFORMER_FIDELITY} --input_path "{input_file}" --bg_upsampler realesrgan --face_upsample --upscale {UPSCALE} --output_path /kaggle/working/tempcodeform --final_video_path "{output_file}"')
    
    for file_path in file_paths:
        input_file = file_path
        output_file = file_path.replace("/kaggle/working/input_videos_raw","/kaggle/working/codeformeupscaled")
        #os.system(f'python inference_codeformer.py -w {CODEFORMER_FIDELITY} --input_path "{input_file}" --bg_upsampler realesrgan --face_upsample --upscale {UPSCALE} --output_path /kaggle/working/tempcodeform --final_video_path "{output_file}"')
        os.system(f'python inference_codeformer.py -w {CODEFORMER_FIDELITY} --fast_access_dir /kaggle/input/codeformer-fast-access --input_path "{input_file}" --bg_upsampler realesrgan --face_upsample --upscale {UPSCALE} --output_path /kaggle/working/tempcodeform --final_video_path "{output_file}"')

timestamps.append(int(time.time()))
all_folder_created.append("Finished Codeformer")

print("FINISHED CODEFORMER!!")

FINISHED CODEFORMER!!


In [ ]:
# Installation of RealESGRAN, needed regardless of further settings
!nvidia-smi
!curl -LO https://github.com/k4yt3x/video2x/releases/download/6.2.0/video2x-linux-ubuntu2204-amd64.deb
!add-apt-repository -y ppa:ubuntuhandbook1/ffmpeg7
!apt-get update
!apt-get install -y libvulkan1 nvidia-driver-535 ffmpeg ./video2x-linux-ubuntu2204-amd64.deb

%env VK_ICD_FILENAMES=/usr/share/vulkan/icd.d/nvidia_icd.json

import os
import shutil
import cv2
os.makedirs("/kaggle/working/realesgranfirstpass",exist_ok=True)
os.makedirs("/kaggle/working/realesgranfirstpass2",exist_ok=True)

print("Starting REALESGRAN")

if firstfilter == "realesgran":
    for file_path in file_paths:
        input_file = file_path
        output_file = file_path.replace("/kaggle/working/input_videos_raw","/kaggle/working/realesgranfirstpass")  
        output_file2 = file_path.replace("/kaggle/working/input_videos_raw","/kaggle/working/realesgranfirstpass2")  
        
        command = [
            "video2x",
        
            "--input",
            f"\"{input_file}\"",
            "--output",
            f"\"{output_file}\"",
            "--processor",
            filter,
        
            # Encoder options
            "--codec",
            codec,
            "--bit-rate",
            str(bitrate),
            "--log-level",
            loglevel,
            "-e",
            f"preset={preset}",
            "-e",
            f"crf={crf}"
        ]
        
        command.extend([
            "--realesrgan-model",
            model,
            "--scaling-factor",
            str(4)
        ])
        
        if pixfmt != "auto":
            command.extend([
                "--pix-fmt",
                pixfmt
            ])
        
        !{' '.join(command)}

        print("FINISHED FIRST PASS REALESGRAN!!")

        timestamps.append(int(time.time()))
        all_folder_created.append("Finished REALESGRAN")

        #Downscale via LANCZOS to get 2x effective scale
        cap = cv2.VideoCapture(output_file)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        cap.release()
    
        new_width = width//2
        new_height = height//2
        # Force even dimensions for x264
        new_width -= new_width % 2
        new_height -= new_height % 2
        
        cmd = (
            f'ffmpeg -i "{output_file}" '
            f'-vf "scale={new_width}:{new_height}:flags=lanczos" '
            f'-r {FPS} '
            f'-c:v {codec} -crf {crf} -preset {preset} -pix_fmt yuv420p '
            f'-c:a aac -b:a 320k -movflags +faststart "{output_file2}"'
        )
            
        os.system(cmd)


        timestamps.append(int(time.time()))
        all_folder_created.append("Finished First pass LANCZOS")

        print("FINISHED FIRST PASS LANCZOS!!")

Wed Dec 17 19:30:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Main Video Upscale, REALEsgran 

In [ ]:
os.makedirs("/kaggle/working/finalupscaled",exist_ok=True)

for file_path in file_paths:
    if firstfilter == "codeformer":
        input_file = file_path.replace("/kaggle/working/input_videos_raw","/kaggle/working/codeformeupscaled")
    elif firstfilter == "realesgran":
        input_file = file_path.replace("/kaggle/working/input_videos_raw","/kaggle/working/realesgranfirstpass2")
    else:
        input_file = file_path
    output_file = file_path.replace("/kaggle/working/input_videos_raw","/kaggle/working/finalupscaled")  

    if scale == 1:
        shutil.copy(input_file, output_file)
        continue
    
    command = [
        "video2x",
    
        "--input",
        f"\"{input_file}\"",
        "--output",
        f"\"{output_file}\"",
        "--processor",
        filter,
    
        # Encoder options
        "--codec",
        codec,
        "--bit-rate",
        str(bitrate),
        "--log-level",
        loglevel,
        "-e",
        f"preset={preset}",
        "-e",
        f"crf={crf}"
    ]
    
    command.extend([
        "--realesrgan-model",
        model,
        "--scaling-factor",
        str(scale)
    ])
    
    if pixfmt != "auto":
        command.extend([
            "--pix-fmt",
            pixfmt
        ])
    
    !{' '.join(command)}

timestamps.append(int(time.time()))
all_folder_created.append("Finished Second Pass REALEsgran")
print("FINISHED SECOND PASS REALESGRAN")

rmtree_safe("/kaggle/working/codeformeupscaled")
rmtree_safe("/kaggle/working/realesgranfirstpass")
rmtree_safe("/kaggle/working/realesgranfirstpass2")

# Final Scale Down, Audio Meging, Higher Compatibility

In [ ]:
import os
import zipfile
import cv2

import subprocess
import math

def get_sanitized_fps(video_path):
    try:
        # Use ffprobe to get the average frame rate
        cmd = [
            'ffprobe', '-v', 'error', '-select_streams', 'v:0',
            '-show_entries', 'stream=avg_frame_rate',
            '-of', 'default=noprint_wrappers=1:nokey=1', video_path
        ]
        fps_string = subprocess.check_output(cmd).decode('utf-8').strip()
        
        # Handle fractions like "30000/1001" or plain "24"
        if '/' in fps_string:
            num, den = map(int, fps_string.split('/'))
            fps = num / den
        else:
            fps = float(fps_string)

        # Snap to nearest standard integer to ensure RIFE compatibility
        # This handles 23.976 -> 24, 29.97 -> 30, 25.4 (jitter) -> 25
        sanitized_fps = round(fps)
        
        # Safety fallback: if detection fails or gives 0, default to 24
        if sanitized_fps <= 0: return 30
        return sanitized_fps

    except Exception as e:
        print(f"Warning: Could not detect FPS for {video_path}, defaulting to 30. Error: {e}")
        return 30



os.makedirs("/kaggle/working/output",exist_ok=True)
for file_path in file_paths:
    input_file = file_path.replace("/kaggle/working/input_videos_raw","/kaggle/working/finalupscaled")
    output_file = file_path.replace("/kaggle/working/input_videos_raw","/kaggle/working/output")
    audio_file = file_path

    #os.system(f'ffmpeg -i "{input_file}" -i "{audio_file}" -y -map 0:v:0 -map 1:a:0? -filter:v "scale=\'if(gt(iw,ih),{TARGET_RES},-1)\':\'if(gt(ih,iw),{TARGET_RES},-1)\':flags=lanczos" -c:v {codec} -crf {crf} -preset {preset} -c:a aac -b:a 320k -movflags +faststart "{output_file}"')

    cap = cv2.VideoCapture(input_file)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    
    # Compute new dimensions (largest side = TARGET_RES)
    if width >= height:
        new_width = TARGET_RES
        new_height = int(height * (TARGET_RES / width))
    else:
        new_height = TARGET_RES
        new_width = int(width * (TARGET_RES / height))
    
    # Force even dimensions for x264
    new_width -= new_width % 2
    new_height -= new_height % 2
    target_fps = int(get_sanitized_fps(input_file))

    cmd = (
        f'ffmpeg -i "{input_file}" '
        f'-fflags +genpts -i "{audio_file}" -y '
        f'-shortest '
        f'-map 0:v:0 -map 1:a:0? '
        f'-vf "scale={new_width}:{new_height}:flags=lanczos,setpts=PTS-STARTPTS" '
        f'-af "asetpts=PTS-STARTPTS" '
        f'-r {target_fps} -vsync cfr ' 
        f'-c:v libx264 -crf {crf} -preset {preset} -pix_fmt yuv420p '
        f'-c:a aac -b:a 320k -movflags +faststart "{output_file}"'
    )
        
    os.system(cmd)

rmtree_safe("/kaggle/working/finalupscaled")

# RIFE

In [ ]:
%cd /content
!git clone https://github.com/hzwer/arXiv2020-RIFE

!mkdir /content/arXiv2020-RIFE/train_log
%cd /content/arXiv2020-RIFE/train_log
!gdown --id 1APIzVeI-4ZZCEuIRE1m6WYfSCaOsi_7_
!7z e RIFE_trained_model_v3.6.zip

%cd /content/arXiv2020-RIFE/
#!gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
!pip install git+https://github.com/rk-exxec/scikit-video.git@numpy_deprecation

!pip install scenedetect[opencv] --upgrade

In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
import os
import subprocess
import tempfile
import logging
import shutil
from scenedetect import detect, ContentDetector, split_video_ffmpeg

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


print(f"RIFE SCALE: {rife_scale}")
def interpolate_video(input_file: str, output_file: str, target_fps: int):
    """
    Interpolates a video to a target FPS by splitting it into scenes,
    running RIFE on each scene, and then concatenating the results.

    Args:
        input_file (str): Path to the source video file.
        output_file (str): Path to save the final interpolated video.
        target_fps (int): The desired output FPS (e.g., 60, 120).
    """
    
    with tempfile.TemporaryDirectory() as temp_dir:
        logging.info(f"Created temporary directory: {temp_dir}")

        logging.info("Detecting scenes...")
        scene_list = detect(input_file, ContentDetector())

        scene_files_to_process = []
        if not scene_list:
            logging.warning("No scene cuts detected. Processing the entire video as a single clip.")
            scene_files_to_process.append(input_file)
        else:
            logging.info(f"Found {len(scene_list)} scenes. Splitting video into temporary files...")
            output_template = os.path.join(temp_dir, 'scene-$SCENE_NUMBER.mp4')
            split_video_ffmpeg(input_file, scene_list, output_file_template=output_template)
            scene_files_to_process = sorted(
                [os.path.join(temp_dir, f) for f in os.listdir(temp_dir) if f.startswith('scene-') and f.endswith('.mp4')],
                key=lambda x: int(x.split('-')[-1].split('.')[0])
            )

        interpolated_files = []
        # The --exp argument is an exponent for 2x interpolation (2^1=2x, 2^2=4x, etc.)
        # To get from ~30fps to ~60fps, we need 2x interpolation (--exp 1)
        # To get from ~30fps to ~120fps, we need 4x interpolation (--exp 2)
        exp_factor = 4 if target_fps > 90 else 2
        logging.info(f"Target FPS is {target_fps}. Using 2^{exp_factor}x interpolation (--exp {exp_factor}).")

        for i, scene_file in enumerate(scene_files_to_process):
            base_name = os.path.basename(scene_file)
            logging.info(f"Interpolating scene {i+1}/{len(scene_files_to_process)}: {base_name}")
            
            interpolated_output_path = os.path.join(temp_dir, f"interpolated_{i+1:04d}.mp4")
            interpolated_files.append(interpolated_output_path)

            command = [
                'python', 'inference_video.py',
                '--exp', str(exp_factor),
                '--video', scene_file,
                '--output', interpolated_output_path,
                '--scale', str(rife_scale)
            ]
            
            result = subprocess.run(command, capture_output=True, text=True, check=False)
            if result.returncode != 0:
                logging.error(f"Error interpolating {base_name}:")
                logging.error(result.stderr)
                return # Stop the process if a scene fails

        logging.info("Concatenating interpolated scenes...")
        concat_list_path = os.path.join(temp_dir, 'concat_list.txt')
        
        # ffmpeg's concat demuxer requires a specific file format
        with open(concat_list_path, 'w') as f:
            for file_path in interpolated_files:
                f.write(f"file '{os.path.abspath(file_path)}'\n")

        concat_command = [
            'ffmpeg',
            '-f', 'concat',
            '-safe', '0', # Allow unsafe file paths (necessary for temp dirs)
            '-i', concat_list_path,
            '-c', 'copy',
            '-y',
            output_file
        ]

        result = subprocess.run(concat_command, capture_output=True, text=True, check=False)
        if result.returncode != 0:
            logging.error("Error during final concatenation:")
            logging.error(result.stderr)
            return

        logging.info(f"Successfully created interpolated video: {output_file}")

    logging.info("Temporary files have been cleaned up.")


os.makedirs("/kaggle/working/rife_interped",exist_ok=True)
for file_path in file_paths:
    output_file = file_path.replace("/kaggle/working/input_videos_raw","/kaggle/working/rife_interped")
    input_file = file_path.replace("/kaggle/working/input_videos_raw","/kaggle/working/output")

    #os.system(f'python inference_video.py --exp {2 if FPS < 90 else 4} --video="{input_file}" --output "{output_file}" --UHD')
    interpolate_video(input_file, output_file, FPS)


timestamps.append(int(time.time()))
all_folder_created.append("Finished RIFE")
print("FINISHED RIFE")

# Add back audio + zip

In [ ]:
os.makedirs("/kaggle/working/output2", exist_ok = True)

for file_path in file_paths:
    input_file = file_path.replace("/kaggle/working/input_videos_raw","/kaggle/working/rife_interped")
    audio_file = file_path.replace("/kaggle/working/input_videos_raw","/kaggle/working/input_videos_raw")
    output_file = file_path.replace("/kaggle/working/input_videos_raw","/kaggle/working/output2")

    #old command
    cmd = (
        f'ffmpeg -i "{input_file}" '
        f'-i "{audio_file}" -y '
        f'-shortest '

        f'-map 0:v:0 -map 1:a:0? '
        f'-c:v libx264 -crf {crf} -preset {preset} -pix_fmt yuv420p '
        f'-c:a aac -b:a 320k -movflags +faststart "{output_file}"'
    )



    cmd = (
        f'ffmpeg -i "{input_file}" '
        
        f'-fflags +genpts -i "{audio_file}" -y '
        f'-shortest '
        f'-map 0:v:0 -map 1:a:0? '
        f'-vf "setpts=PTS-STARTPTS" '
        f'-af "asetpts=PTS-STARTPTS" '
        f'-r {FPS} '
        f'-c:v libx264 -crf {crf} -preset {preset} -pix_fmt yuv420p '
        f'-c:a aac -b:a 320k -movflags +faststart "{output_file}"'
    )


    os.system(cmd)


def zip_folder(folder_path, output_zip):
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))


zip_folder('/kaggle/working/output2', '/kaggle/working/output_videos.zip')

timestamps.append(int(time.time()))
all_folder_created.append("Finished Final scale down")
print("FINISHED SCALE DOWN")
rmtree_safe("/kaggle/working/output")
rmtree_safe("/kaggle/working/finalupscaled")
rmtree_safe("/kaggle/working/rife_interped")

In [ ]:
INPUT_VIDEO_FOLDER = "/kaggle/working/output2"

from PIL import Image, ImageFilter, ImageOps
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
import zipfile
import os
import shutil


def vibrance(img, vib):
    hsv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2HSV).astype("float32").copy()

    h, s, v = cv2.split(hsv)
    s = s + (255 - s) * vib * (1 - (s / 255))
    s = np.clip(s, 0, 255)

    vibrant_hsv = cv2.merge([h, s, v])
    vibrant_img = cv2.cvtColor(vibrant_hsv.astype("uint8"), cv2.COLOR_HSV2RGB)

    return Image.fromarray(vibrant_img)

class LUT:
    def __init__(self,path):
        with open(path, 'r') as f:
            lines = f.readlines()
            
        # Extract LUT size and data
        size = 0
        table = []
        
        for line in lines:
            if line.startswith('#') or not line.strip():
                continue
                
            if 'LUT_3D_SIZE' in line:
                size = int(line.split()[1])
                continue
                
            if 'DOMAIN_' in line:
                continue

            parts = line.split()
            if len(parts) == 3:
                table.extend([float(p) for p in parts])

        if size == 0:
            raise ValueError("Could not find LUT_3D_SIZE in the cube file.")

        self.size, self.table = size, table

        self.lut = ImageFilter.Color3DLUT(size, table)

    def apply_pil(self,img, strength=1):
        return Image.blend(img, img.filter(self.lut), strength)

class UnsharpMask:
    def __init__(self, radius=10, percent=100, threshold=3):
        self.radius = radius
        self.percent = percent
        self.threshold = threshold
    
    def apply_pil(self, img):
        s = max(img.size)/1000
        return img.filter(ImageFilter.UnsharpMask(radius=self.radius*s, percent=self.percent, threshold=self.threshold))
        
class UnsharpMask2:
    def __init__(self, radius=3, percent=100, threshold=5):
        self.radius = radius
        self.percent = percent
        self.threshold = threshold
    
    def apply_pil(self, img):
        s = max(img.size)/1000
        ycbcr = img.convert("YCbCr")
        (y, cb, cr) = ycbcr.split()
        
        sharpened_y = y.filter(ImageFilter.UnsharpMask(radius=self.radius*s, percent=self.percent, threshold=self.threshold))
        
        final_ycbcr = Image.merge("YCbCr", (sharpened_y, cb, cr))
        
        final_rgb = final_ycbcr.convert("RGB")
        
        return final_rgb


merge_mertens = cv2.createMergeMertens() 
def hdrify(img_pil):
    img = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    img_under = cv2.addWeighted(img, 0.5, np.zeros(img.shape, img.dtype), 0, 0) 
    img_over = cv2.addWeighted(img, 1.5, np.zeros(img.shape, img.dtype), 0, 0) 

    img_list = [img_under, img, img_over] 

    res_mertens = merge_mertens.process(img_list) 

    return Image.fromarray(cv2.cvtColor(np.clip(res_mertens*255, 0, 255).astype('uint8'), cv2.COLOR_BGR2RGB))


class HDR:
    def __init__(self):
        self.lut = LUT("/kaggle/input/colorluts/pop.cube")

    def apply_pil(self, raw_img):
        img = raw_img.copy()
        img = self.lut.apply_pil(img, 0.6)

        #enh = ImageEnhance.Sharpness(img)
        #img = enh.enhance(2)

        img = img.filter(ImageFilter.DETAIL)
        #img = img.filter(ImageFilter.DETAIL)
        #img = vibrance(img, 0.1)
        #img = unsharp.apply_pil(img)

        img = hdrify(img)
   
        return img



def apply_hdr_video(input_video, output_video):
    hdrobj = HDR()

    def transform_frame(get_frame, t):
        current_frame = get_frame(t)

        pil_img = Image.fromarray(current_frame)
        processed_pil = hdrobj.apply_pil(pil_img)

        return np.array(processed_pil)

    def filter_frame(frame):
        return np.array(hdrobj.apply_pil(Image.fromarray(frame)))


    clip = VideoFileClip(input_video)

    #processed_clip = clip.transform(transform_frame)
    processed_clip = clip.fl_image(filter_frame)

    processed_clip.write_videofile(output_video, audio=True, threads=4)


#apply_hdr_video("input.mp4","output.mp4")
from pathlib import Path

input_folder = Path(INPUT_VIDEO_FOLDER)
output_folder = Path("/kaggle/working/output_videosHDR")

output_folder.mkdir(exist_ok=True)

for file in input_folder.iterdir():
    if file.is_file() and file.suffix.lower() in [".mp4", ".mov", ".mkv", ".avi"]:
        output_path = output_folder / file.name
        apply_hdr_video(str(file), str(output_path))
        print(f"Processed: {file.name}")



def zip_folder(folder_path, output_zip):
    # Create a ZipFile object in write mode
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through all files and subfolders in the folder
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create the full path to the file
                file_path = os.path.join(root, file)
                # Add file to the zip file with relative path
                zipf.write(file_path, os.path.relpath(file_path, folder_path))


zip_folder('/kaggle/working/output_videosHDR', '/kaggle/working/output_videosHDR.zip')

# Notification

In [ ]:
if ntfy_channel is not None:
    import requests
    requests.post(f"https://ntfy.sh/{ntfy_channel}",
    data="Finished upscaling videos! 😀".encode(encoding='utf-8'))

timestamps.append(int(time.time()))
all_folder_created.append("Finished Everything")


print("---DEBUG TIMESTAMPS---:")
print(all_folder_created)
print(timestamps)
print(list(zip(all_folder_created,timestamps)))